##Zimnat Insurance Recommendation Challenge
Can you predict which insurance products existing clients will want next?
###Baseline with catboost

In [ ]:
# Memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

In [ ]:
import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

### Get data

In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import datetime
import gc
gc.collect()

233

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/drive/My Drive/Train_Zimnat.csv')
test = pd.read_csv('/content/drive/My Drive/Test_Zimnat.csv')
submission = pd.read_csv('/content/drive/My Drive/SampleSubmission_Zimnat.csv')

In [ ]:
train.shape, test.shape, submission.shape

((29132, 29), (10000, 29), (210000, 2))

In [ ]:
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,01/02/2019,F,M,1987,1X1H,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,01/06/2019,F,M,1981,UAOD,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,01/06/2013,M,U,1991,748L,QZYX,90QI,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,2S9E81J,01/08/2019,M,M,1990,1X1H,BP09,56SI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
4,BHDYVFT,01/08/2019,M,M,1990,748L,NO3L,T4MS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


### New format

In [ ]:
X_train = []
X_train_columns = train.columns
c = 0
for v in train.values:
  info = v[:8]
  binary = v[8:]
  index = [k for k, i in enumerate(binary) if i == 1]
  for i in index:
    c+=1
    for k in range(len(binary)):
      if k == i:
        binary_transformed = list(copy.copy(binary))
        binary_transformed[i] = 0
        X_train.append(list(info) + binary_transformed + [X_train_columns[8+k]] + [c])

X_train = pd.DataFrame(X_train)
X_train.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred', 'ID2']

In [ ]:
X_test = []
true_values = []
c = 0
for v in test.values:
  c += 1
  info = v[:8]
  binary = v[8:]
  index = [k for k, i in enumerate(binary) if i == 1]
  X_test.append(list(info) + list(binary) + [c])
  for k in test.columns[8:][index]:
    true_values.append(v[0] + ' X ' + k)

X_test = pd.DataFrame(X_test)
X_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'ID2']

### Transform data

In [ ]:
features_train = []
features_test = []
columns = []

append_features = ['P5DA', 'RIBP', '8NN1', '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 
'N2MW', 'AHXO','BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 
'ECY3', 'ID', 'ID2', 'join_date', 'sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code',
'birth_year']
for v in append_features:
  features_train.append(X_train[v].values.reshape(-1, 1))
  features_test.append(X_test[v].values.reshape(-1, 1))
  columns.append(np.array([v]))

y_train = X_train[['product_pred']]

In [ ]:
features_train = np.concatenate(features_train, axis=1)
features_test = np.concatenate(features_test, axis=1)
columns = np.concatenate(np.array(columns))

X_train = pd.DataFrame(features_train)
X_train.columns = columns
X_test = pd.DataFrame(features_test)
X_test.columns = columns

### New features

In [ ]:
X_train['join_date'] = pd.to_datetime(X_train['join_date'])
X_train['year'] = X_train['join_date'].dt.year
X_train['weekofyear'] = X_train['join_date'].dt.weekofyear
X_train['month'] = X_train['join_date'].dt.month
X_train['dayofweek'] = X_train['join_date'].dt.dayofweek
X_train['weekend'] = (X_train.join_date.dt.weekday >=5).astype(int)
X_train['hour'] = X_train['join_date'].dt.hour 
X_train['quarter'] = X_train['join_date'].dt.quarter
X_train['is_month_start'] = X_train['join_date'].dt.is_month_start
X_train['month_diff'] = ((datetime.datetime.today() - X_train['join_date']).dt.days)//30
#X_train['month_diff'] += X_train['month_lag']
X_train.drop('join_date', axis=1, inplace=True)

X_test['join_date'] = pd.to_datetime(X_test['join_date'])
X_test['year'] = X_test['join_date'].dt.year
X_test['weekofyear'] = X_test['join_date'].dt.weekofyear
X_test['month'] = X_test['join_date'].dt.month
X_test['dayofweek'] = X_test['join_date'].dt.dayofweek
X_test['weekend'] = (X_test.join_date.dt.weekday >=5).astype(int)
X_test['hour'] = X_test['join_date'].dt.hour 
X_test['quarter'] = X_test['join_date'].dt.quarter
X_test['is_month_start'] = X_test['join_date'].dt.is_month_start
X_test['month_diff'] = ((datetime.datetime.today() - X_test['join_date']).dt.days)//30
#X_test['month_diff'] += X_test['month_lag']
X_test.drop('join_date', axis=1, inplace=True)

X_train['customer_age_as_at_join_date'] = X_train['year'] - X_train['birth_year']
X_test['customer_age_as_at_join_date'] = X_test['year'] - X_test['birth_year']

In [ ]:
# additional features
X_train['customer_month_diff_feature'] = X_train['customer_age_as_at_join_date']*X_train['month_diff']
X_train['customer_month_diff_ratio'] = X_train['customer_age_as_at_join_date']/X_train['month_diff']

X_test['customer_month_diff_feature'] = X_test['customer_age_as_at_join_date']*X_test['month_diff']
X_test['customer_month_diff_ratio'] = X_test['customer_age_as_at_join_date']/X_test['month_diff']

gc.collect()

111

In [ ]:
X_train.head()

,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,ID,ID2,sex,marital_status,branch_code,occupation_code,occupation_category_code,birth_year,year,weekofyear,month,dayofweek,weekend,hour,quarter,is_month_start,month_diff,customer_age_as_at_join_date,customer_month_diff_feature,customer_month_diff_ratio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4WKQSBB,1,F,M,1X1H,2A7I,T4MS,1987,2019.0,1.0,1.0,2.0,0,0.0,1.0,False,20.0,32,640,1.6
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4WKQSBB,2,F,M,1X1H,2A7I,T4MS,1987,2019.0,1.0,1.0,2.0,0,0.0,1.0,False,20.0,32,640,1.6
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,CP5S02H,3,F,M,UAOD,2A7I,T4MS,1981,2019.0,1.0,1.0,6.0,1,0.0,1.0,False,20.0,38,760,1.9
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,CP5S02H,4,F,M,UAOD,2A7I,T4MS,1981,2019.0,1.0,1.0,6.0,1,0.0,1.0,False,20.0,38,760,1.9
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,2YKDILJ,5,M,U,748L,QZYX,90QI,1991,2013.0,1.0,1.0,6.0,1,0.0,1.0,False,93.0,22,2046,0.236559


In [ ]:
X_test.head()

,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,ID,ID2,sex,marital_status,branch_code,occupation_code,occupation_category_code,birth_year,year,weekofyear,month,dayofweek,weekend,hour,quarter,is_month_start,month_diff,customer_age_as_at_join_date,customer_month_diff_feature,customer_month_diff_ratio
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,F86J5PC,1,M,M,94KC,DZRV,90QI,1984,2018.0,2.0,1.0,4.0,0,0.0,1.0,False,32.0,34,1088,1.0625
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,H6141K3,2,M,M,1X1H,J9SY,90QI,1996,2019.0,2.0,1.0,3.0,0,0.0,1.0,False,20.0,23,460,1.15
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,RBAYUXZ,3,F,W,UAOD,2A7I,T4MS,1968,2020.0,1.0,1.0,2.0,0,0.0,1.0,True,8.0,52,416,6.5
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,KCBILBQ,4,M,M,94KC,2A7I,T4MS,1989,2019.0,1.0,1.0,2.0,0,0.0,1.0,False,20.0,30,600,1.5
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,LSEC1ZJ,5,F,M,UAOD,0KID,T4MS,1982,2020.0,1.0,1.0,3.0,0,0.0,1.0,False,8.0,38,304,4.75


In [ ]:
X_train.shape, X_test.shape

((66353, 41), (10000, 41))

In [ ]:
#X_train['date1'] = X_train['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
#X_train['date2'] = X_train['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
#X_train['date3'] = X_train['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
#X_train.drop('join_date', axis=1, inplace=True)

#X_test['date1'] = X_test['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
#X_test['date2'] = X_test['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
#X_test['date3'] = X_test['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
#X_test.drop('join_date', axis=1, inplace=True)

#X_train['date_diff'] = X_train['date3'] - X_train['birth_year']
#X_test['date_diff'] = X_test['date3'] - X_test['birth_year']

In [ ]:
X_train.to_csv("X_train.csv")

In [ ]:
X_test.to_csv("X_test.csv")

# Groupby all the columns using "birth_year" feature

In [ ]:
bi_yr = X_train.groupby('birth_year').mean()
#bi_yr.drop('ID', 'ID2', 1 #, inplace = True)
cols = []
for i in bi_yr.columns:
    if i != 'birth_year':
        bi_yr[i+'_reg_mean_all'] = bi_yr[i]
        bi_yr.drop(i, 1, inplace = True)
        cols.append(i+'_reg_mean_all')

X_train = pd.merge(X_train, bi_yr, on='birth_year', how = 'left')
for col in cols: X_train[col+'_freq'] = X_train[col].map(X_train[col].value_counts().to_dict())/len(X_train)

In [ ]:
bi_yr = X_test.groupby('birth_year').mean()
#bi_yr.drop('ID', 'ID2', 1) #, inplace = True)
cols = []
for i in bi_yr.columns:
    if i != 'birth_year':
        bi_yr[i+'_reg_mean_all'] = bi_yr[i]
        bi_yr.drop(i, 1, inplace = True)
        cols.append(i+'_reg_mean_all')

X_test = pd.merge(X_test, bi_yr, on='birth_year', how = 'left')
for col in cols: X_test[col+'_freq'] = X_test[col].map(X_test[col].value_counts().to_dict())/len(X_test)

In [ ]:
X_train.head()

,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,ID,ID2,sex,marital_status,branch_code,occupation_code,occupation_category_code,birth_year,year,weekofyear,month,dayofweek,weekend,hour,quarter,is_month_start,month_diff,customer_age_as_at_join_date,customer_month_diff_feature,customer_month_diff_ratio,year_reg_mean_all,weekofyear_reg_mean_all,month_reg_mean_all,dayofweek_reg_mean_all,weekend_reg_mean_all,hour_reg_mean_all,quarter_reg_mean_all,is_month_start_reg_mean_all,month_diff_reg_mean_all,year_reg_mean_all_freq,weekofyear_reg_mean_all_freq,month_reg_mean_all_freq,dayofweek_reg_mean_all_freq,weekend_reg_mean_all_freq,hour_reg_mean_all_freq,quarter_reg_mean_all_freq,is_month_start_reg_mean_all_freq,month_diff_reg_mean_all_freq
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4WKQSBB,1,F,M,1X1H,2A7I,T4MS,1987,2019.0,1.0,1.0,2.0,0,0.0,1.0,False,20.0,32,640,1.6,2017.734328,2.389339,1.004264,3.035821,0.255864,0.0,1.000853,0.057143,35.287846,0.035341,0.035341,0.035341,0.035341,0.035341,1.0,0.035341,0.035341,0.035341
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4WKQSBB,2,F,M,1X1H,2A7I,T4MS,1987,2019.0,1.0,1.0,2.0,0,0.0,1.0,False,20.0,32,640,1.6,2017.734328,2.389339,1.004264,3.035821,0.255864,0.0,1.000853,0.057143,35.287846,0.035341,0.035341,0.035341,0.035341,0.035341,1.0,0.035341,0.035341,0.035341
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,CP5S02H,3,F,M,UAOD,2A7I,T4MS,1981,2019.0,1.0,1.0,6.0,1,0.0,1.0,False,20.0,38,760,1.9,2017.809274,2.279090,1.003500,2.966317,0.247594,0.0,1.000875,0.076115,34.386264,0.034452,0.034452,0.034452,0.034452,0.034452,1.0,0.034452,0.034452,0.034452
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,CP5S02H,4,F,M,UAOD,2A7I,T4MS,1981,2019.0,1.0,1.0,6.0,1,0.0,1.0,False,20.0,38,760,1.9,2017.809274,2.279090,1.003500,2.966317,0.247594,0.0,1.000875,0.076115,34.386264,0.034452,0.034452,0.034452,0.034452,0.034452,1.0,0.034452,0.034452,0.034452
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,2YKDILJ,5,M,U,748L,QZYX,90QI,1991,2013.0,1.0,1.0,6.0,1,0.0,1.0,False,93.0,22,2046,0.236559,2017.922362,2.457200,1.000000,3.096218,0.278699,0.0,1.000000,0.078965,33.009954,0.022712,0.022712,0.683601,0.022712,0.022712,1.0,0.695779,0.022712,0.022712


In [ ]:
X_test.head()

,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,ID,ID2,sex,marital_status,branch_code,occupation_code,occupation_category_code,birth_year,year,weekofyear,month,dayofweek,weekend,hour,quarter,is_month_start,month_diff,customer_age_as_at_join_date,customer_month_diff_feature,customer_month_diff_ratio,year_reg_mean_all,weekofyear_reg_mean_all,month_reg_mean_all,dayofweek_reg_mean_all,weekend_reg_mean_all,hour_reg_mean_all,quarter_reg_mean_all,is_month_start_reg_mean_all,month_diff_reg_mean_all,year_reg_mean_all_freq,weekofyear_reg_mean_all_freq,month_reg_mean_all_freq,dayofweek_reg_mean_all_freq,weekend_reg_mean_all_freq,hour_reg_mean_all_freq,quarter_reg_mean_all_freq,is_month_start_reg_mean_all_freq,month_diff_reg_mean_all_freq
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,F86J5PC,1,M,M,94KC,DZRV,90QI,1984,2018.0,2.0,1.0,4.0,0,0.0,1.0,False,32.0,34,1088,1.0625,2017.747788,2.311947,1.0,2.988938,0.280973,0.0,1.0,0.081858,35.139381,0.0452,0.0452,0.9314,0.0452,0.0452,1.0,0.9413,0.0452,0.0452
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,H6141K3,2,M,M,1X1H,J9SY,90QI,1996,2019.0,2.0,1.0,3.0,0,0.0,1.0,False,20.0,23,460,1.15,2018.558140,2.093023,1.0,2.918605,0.255814,0.0,1.0,0.069767,25.313953,0.0086,0.0086,0.9314,0.0086,0.0086,1.0,0.9413,0.0086,0.0086
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,RBAYUXZ,3,F,W,UAOD,2A7I,T4MS,1968,2020.0,1.0,1.0,2.0,0,0.0,1.0,True,8.0,52,416,6.5,2017.372197,2.968610,1.0,3.022422,0.269058,0.0,1.0,0.094170,39.704036,0.0223,0.0223,0.9314,0.0223,0.0223,1.0,0.9413,0.0223,0.0223
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,KCBILBQ,4,M,M,94KC,2A7I,T4MS,1989,2019.0,1.0,1.0,2.0,0,0.0,1.0,False,20.0,30,600,1.5,2017.828488,2.023256,1.0,3.113372,0.273256,0.0,1.0,0.061047,34.130814,0.0344,0.0344,0.9314,0.0344,0.0344,1.0,0.9413,0.0344,0.0344
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,LSEC1ZJ,5,F,M,UAOD,0KID,T4MS,1982,2020.0,1.0,1.0,3.0,0,0.0,1.0,False,8.0,38,304,4.75,2017.738041,2.220957,1.0,2.979499,0.239180,0.0,1.0,0.079727,35.250569,0.0439,0.0439,0.9314,0.0439,0.0439,1.0,0.9413,0.0439,0.0439


In [ ]:
X_train.shape, X_test.shape

((66353, 59), (10000, 59))

In [ ]:
categorical_features=[feature for feature in X_train.columns if X_train[feature].dtype=='O']

categorical_features

['P5DA',
 'RIBP',
 '8NN1',
 '7POT',
 '66FJ',
 'GYSR',
 'SOP4',
 'RVSZ',
 'PYUQ',
 'LJR9',
 'N2MW',
 'AHXO',
 'BSTQ',
 'FM3X',
 'K6QO',
 'QBOL',
 'JWFN',
 'JZ9D',
 'J9JW',
 'GHYX',
 'ECY3',
 'ID',
 'ID2',
 'sex',
 'marital_status',
 'branch_code',
 'occupation_code',
 'occupation_category_code',
 'birth_year',
 'customer_age_as_at_join_date',
 'customer_month_diff_feature',
 'customer_month_diff_ratio']

In [ ]:
numerical_features=[feature for feature in X_train.columns if X_train[feature].dtype !='O']

numerical_features

['year',
 'weekofyear',
 'month',
 'dayofweek',
 'weekend',
 'hour',
 'quarter',
 'is_month_start',
 'month_diff',
 'year_reg_mean_all',
 'weekofyear_reg_mean_all',
 'month_reg_mean_all',
 'dayofweek_reg_mean_all',
 'weekend_reg_mean_all',
 'hour_reg_mean_all',
 'quarter_reg_mean_all',
 'is_month_start_reg_mean_all',
 'month_diff_reg_mean_all',
 'year_reg_mean_all_freq',
 'weekofyear_reg_mean_all_freq',
 'month_reg_mean_all_freq',
 'dayofweek_reg_mean_all_freq',
 'weekend_reg_mean_all_freq',
 'hour_reg_mean_all_freq',
 'quarter_reg_mean_all_freq',
 'is_month_start_reg_mean_all_freq',
 'month_diff_reg_mean_all_freq']

In [ ]:
X_train.shape, X_test.shape

((66353, 59), (10000, 59))

### Change types

In [ ]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)
y_train = y_train.fillna(0)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data = X_train.append(X_test)
for v in ['sex','marital_status','branch_code','occupation_code','occupation_category_code','birth_year',
          'customer_age_as_at_join_date','customer_month_diff_feature','customer_month_diff_ratio',]:
    data.loc[:,v] = le.fit_transform(data.loc[:,v])
X_train = data[:X_train.shape[0]]
X_test = data[-X_test.shape[0]:]

In [ ]:
le.fit(y_train.iloc[:,0])
y_train = pd.DataFrame(le.transform(y_train.iloc[:,0]))
y_train.columns = ['target']

### Model

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(n_estimators=3000)
model.fit(X_train.drop(columns=['ID', 'ID2']), y_train,
          cat_features=['sex','marital_status','branch_code','occupation_code',
                        'occupation_category_code','birth_year','customer_age_as_at_join_date', 
                        'customer_month_diff_feature','customer_month_diff_ratio'],
          early_stopping_rounds=200,verbose=100)

Learning rate set to 0.054559
0:	learn: 2.3040819	total: 15.2s	remaining: 8h 26m 44s
100:	learn: 0.4735968	total: 29m 32s	remaining: 9h 15m 17s
200:	learn: 0.4186033	total: 1h 3m 47s	remaining: 9h 30m 58s
300:	learn: 0.3878471	total: 1h 36m 51s	remaining: 9h 6m 40s
400:	learn: 0.3754549	total: 2h 12m 44s	remaining: 8h 49m 17s
500:	learn: 0.3646385	total: 2h 47m 58s	remaining: 8h 22m 34s
600:	learn: 0.3581269	total: 3h 24m 20s	remaining: 7h 55m 38s
700:	learn: 0.3519039	total: 4h 2s	remaining: 7h 24m 49s
800:	learn: 0.3467551	total: 4h 36m 31s	remaining: 6h 53m 54s
900:	learn: 0.3413305	total: 5h 13m 32s	remaining: 6h 22m 26s
1000:	learn: 0.3367288	total: 5h 50m 10s	remaining: 5h 49m 28s
1100:	learn: 0.3314011	total: 6h 26m 5s	remaining: 5h 15m 15s
1200:	learn: 0.3260407	total: 7h 2m 20s	remaining: 4h 40m 58s
1300:	learn: 0.3214630	total: 7h 37m 58s	remaining: 4h 6m 3s


In [ ]:
proba = model.predict_proba(X_test.drop(columns=['ID','ID2'], axis=1))
y_test = pd.DataFrame(proba)
y_test.columns = le.inverse_transform(y_test.columns)

### Submit

In [ ]:
answer_mass = []
for i in range(X_test.shape[0]):
  id = X_test['ID'].iloc[i]
  for c in y_test.columns:
    answer_mass.append([id + ' X ' + c, y_test[c].iloc[i]])

df_answer = pd.DataFrame(answer_mass)
df_answer.columns = ['ID X PCODE', 'Label']
for i in range(df_answer.shape[0]):
  if df_answer['ID X PCODE'].iloc[i] in true_values:
    df_answer['Label'].iloc[i] = 1.0

In [ ]:
df_answer.reset_index(drop=True, inplace=True)
df_answer.to_csv('submission1.csv', index=False)

In [ ]:
df_answer.head()

In [ ]:
df_answer.shape

In [ ]:
from google.colab import files
files.download('submission1.csv') 

#NOTE: 

In [ ]:
# submission1__1.csv with 1000 n_estimators === 0.0278955446408729 LB
# submission1__2_.csv with 2000 n_estimators === 0.027731205009537 LB
# submission1__3_.csv with 3000 n_estimators === 0.0277399057489101 LB 